In [ ]:
import os
import gradio as gr
from typing import TypedDict, List, Annotated
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
# =====================================================
# STATE
# =====================================================
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation"]
    city: str
    interests: List[str]
    weather: str
    budget: str
    days: int
    persona: str
    pace: str
    itinerary: str
# =====================================================
# LLM
# =====================================================
llm = ChatGroq(
    temperature=0.35,
    groq_api_key="",
    model_name="llama-3.3-70b-versatile"
)
# =====================================================
# PROMPT (HEAVYWEIGHT)
# =====================================================
itinerary_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
You are a senior travel planner with real-world experience.

Create a {days}-day travel itinerary for {city}.

User profile:
- Interests: {interests}
- Weather: {weather}
- Budget: {budget}
- Travel persona: {persona}
- Travel pace: {pace}

Rules:
- Split itinerary DAY-WISE
- Each day must have Morning / Afternoon / Evening
- Suggest FOOD and REST breaks
- Prefer nearby locations together
- Avoid tourist traps when possible
- Adapt intensity based on travel pace
- Use clean Markdown + emojis
- Be realistic, not fantasy travel
"""
    ),
    ("human", "Create the itinerary.")
])
# =====================================================
# HELPERS
# =====================================================
def parse_interests(text: str) -> List[str]:
    return [i.strip() for i in text.split(",") if i.strip()]

def enrich_interests(interests: List[str], weather: str, persona: str) -> List[str]:
    if weather == "Rainy":
        interests.append("indoor experiences")
    if weather == "Hot":
        interests.append("early morning sightseeing")
    if persona == "Family":
        interests.append("kid-friendly places")
    if persona == "Solo":
        interests.append("local culture exploration")
    return interests

def generate_itinerary(state: PlannerState) -> str:
    try:
        response = llm.invoke(
            itinerary_prompt.format_messages(
                city=state["city"],
                interests=", ".join(state["interests"]),
                weather=state["weather"],
                budget=state["budget"],
                days=state["days"],
                persona=state["persona"],
                pace=state["pace"]
            )
        )
        state["messages"].append(AIMessage(content=response.content))
        state["itinerary"] = response.content
        return response.content

    except Exception as e:
        return f"⚠️ Generation failed: {str(e)}"
# =====================================================
# MAIN FUNCTION
# =====================================================
def travel_planner(
    city: str,
    interests: str,
    days: int,
    weather: str,
    budget: str,
    persona: str,
    pace: str
):
    if not city.strip():
        return "❌ City is required"
    if not interests.strip():
        return "❌ Enter at least one interest"

    parsed = parse_interests(interests)
    enriched = enrich_interests(parsed, weather, persona)

    state: PlannerState = {
        "messages": [],
        "city": city.strip(),
        "interests": enriched,
        "weather": weather,
        "budget": budget,
        "days": days,
        "persona": persona,
        "pace": pace,
        "itinerary": ""
    }

    return generate_itinerary(state)


# =====================================================
# UI
# =====================================================
with gr.Blocks(theme="freddyaboulton/dracula") as app:
    gr.Markdown("# 🌍 AI Travel Planner")
    gr.Markdown("Multi-day • Persona-aware • Budget-smart • LLM-powered ✨")

    with gr.Row():
        city = gr.Textbox(label="🏙️ City", placeholder="Tokyo")
        interests = gr.Textbox(label="🎯 Interests", placeholder="food, history, nightlife")

    with gr.Row():
        days = gr.Slider(1, 5, step=1, value=2, label="📅 Number of Days")
        weather = gr.Dropdown(["Sunny", "Rainy", "Hot", "Cold"], value="Sunny", label="🌦️ Weather")

    with gr.Row():
        budget = gr.Radio(["Budget", "Mid-range", "Luxury"], value="Mid-range", label="💰 Budget")
        persona = gr.Dropdown(["Solo", "Couple", "Family", "Friends"], value="Solo", label="🧍 Travel Persona")

    pace = gr.Radio(["Relaxed", "Balanced", "Fast"], value="Balanced", label="⏱️ Travel Pace")

    generate = gr.Button("🚀 Generate Trip Plan")
    output = gr.Markdown()

    generate.click(
        travel_planner,
        inputs=[city, interests, days, weather, budget, persona, pace],
        outputs=output
    )

app.launch(share=True)

C:\Users\SUBHADEEP PAN\AppData\Roaming\Python\Python313\site-packages\gradio\blocks.py:1195: UserWarning: Cannot load freddyaboulton/dracula. Caught Exception: The space freddyaboulton/dracula does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://1ea742b46b2b63e2f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
